In [1]:
import lightgbm
import pandas as pd
import numpy as np

In [2]:
DATA_PATH = r"../../../../../Data/Elo Merchant Category Recommendation/"

In [3]:
df_train = pd.read_csv(DATA_PATH + r'train.csv', header='infer', sep=',')

In [4]:
df_test = pd.read_csv(DATA_PATH + r'test.csv', header='infer', sep=',')

In [5]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [6]:
df_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [7]:
df_train.loc[:,"first_active_month"] = df_train.loc[:,"first_active_month"].apply(lambda x: (pd.to_datetime(x,format = "%Y-%m")-pd.to_datetime("1970-01-01")).days)

In [8]:
df_test.loc[:,"first_active_month"] = df_test.loc[:,"first_active_month"].apply(lambda x: (pd.to_datetime(x,format = "%Y-%m")-pd.to_datetime("1970-01-01")).days)

In [9]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,17318,C_ID_92a2005557,5,2,1,-0.820283
1,17167,C_ID_3d0044924f,4,1,0,0.392913
2,17014,C_ID_d639edf6cd,2,2,0,0.688056
3,17410,C_ID_186d6a6901,4,3,0,0.142495
4,17471,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [10]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,17318,C_ID_92a2005557,5,2,1,-0.820283
1,17167,C_ID_3d0044924f,4,1,0,0.392913
2,17014,C_ID_d639edf6cd,2,2,0,0.688056
3,17410,C_ID_186d6a6901,4,3,0,0.142495
4,17471,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [11]:
part = np.random.rand(len(df_train))<0.7

In [12]:
part

array([False,  True,  True, ...,  True,  True,  True], dtype=bool)

In [13]:
df_valid = df_train[~part]

In [14]:
df_train = df_train[part]

In [15]:
df_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,17257.0,C_ID_0ab67a22ab,3,3,1
1,17167.0,C_ID_130fd0cbdd,2,3,0
2,17379.0,C_ID_b709037bc5,5,1,1
3,17501.0,C_ID_d27d835a9f,2,1,0
4,16770.0,C_ID_2b5e3df5c2,5,1,1


In [16]:
train_card_id = df_train.loc[:,'card_id']
valid_card_id = df_valid.loc[:,'card_id']
test_card_id = df_test.loc[:,'card_id']
x_train = df_train.loc[:,~df_train.columns.isin(['target','card_id'])]
x_valid = df_valid.loc[:,~df_valid.columns.isin(['target','card_id'])]
x_test = df_test.loc[:,~df_test.columns.isin(['card_id'])]
y_train = df_train.loc[:,'target']
y_valid = df_valid.loc[:,'target']

In [17]:
y_valid.head(7)

0    -0.820283
10   -1.029562
13    0.093633
14    1.597750
16    1.132236
17    0.318062
21    4.747602
Name: target, dtype: float64

In [18]:
lightgbm_train = lightgbm.Dataset(x_train,y_train)

In [19]:
lightgbm_eval = lightgbm.Dataset(x_valid,y_valid)

In [20]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 2
}

In [21]:
lightgbm_trained = lightgbm.train(params,
                lightgbm_train,
                num_boost_round=20,
                valid_sets=lightgbm_eval,
                early_stopping_rounds=5)

[1]	valid_0's l2: 14.7499	valid_0's l1: 1.58339
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 14.7488	valid_0's l1: 1.58348
[3]	valid_0's l2: 14.7422	valid_0's l1: 1.58337
[4]	valid_0's l2: 14.7371	valid_0's l1: 1.58336
[5]	valid_0's l2: 14.7363	valid_0's l1: 1.58347
[6]	valid_0's l2: 14.7315	valid_0's l1: 1.58362
[7]	valid_0's l2: 14.727	valid_0's l1: 1.58383
[8]	valid_0's l2: 14.7264	valid_0's l1: 1.58403
[9]	valid_0's l2: 14.7258	valid_0's l1: 1.58422
Early stopping, best iteration is:
[4]	valid_0's l2: 14.7371	valid_0's l1: 1.58336


In [22]:
y_pred = lightgbm_trained.predict(x_test,num_iteration=lightgbm_trained.best_iteration)

In [23]:
pd.Series(y_pred)

0        -0.446822
1        -0.448329
2        -0.366807
3        -0.347010
4        -0.446596
5        -0.375431
6        -0.345518
7        -0.408386
8        -0.395413
9        -0.462577
10       -0.425286
11       -0.341917
12       -0.401809
13       -0.438452
14       -0.449049
15       -0.344041
16       -0.350055
17       -0.359134
18       -0.355673
19       -0.459937
20       -0.444397
21       -0.391109
22       -0.438452
23       -0.366807
24       -0.401809
25       -0.430209
26       -0.400463
27       -0.409521
28       -0.384058
29       -0.409521
            ...   
123593   -0.401809
123594   -0.355712
123595   -0.343263
123596   -0.409521
123597   -0.409521
123598   -0.348819
123599   -0.357835
123600   -0.445041
123601   -0.360805
123602   -0.441654
123603   -0.338519
123604   -0.347010
123605   -0.406813
123606   -0.375431
123607   -0.341917
123608   -0.409521
123609   -0.449303
123610   -0.384541
123611   -0.436624
123612   -0.400463
123613   -0.444228
123614   -0.

In [24]:
result = pd.concat([test_card_id,pd.Series(y_pred)],axis = 1,keys = ["card_id","target"])

In [25]:
result.columns

Index(['card_id', 'target'], dtype='object')

In [26]:
result.to_csv(DATA_PATH + r'result.csv', header='infer', sep=',', index=False)